Multisent using Naive bayes

In [1]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import xgboost, numpy, string
import pandas as pd
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

#출처
#https://www.analyticsvidhya.com/blog/2018/04/a-comprehensive-guide-to-understand-and-implement-text-classification-in-python/

Using TensorFlow backend.


In [2]:
#train, test, valid 데이터 read
#각 데이터는 xxx파일에 저장되어 있습니다.
#happy의 경우 happy1만을 세팅한 값입니다. 추후 변경 예정
train_data = pd.read_csv("four_train_data.csv")
test_data = pd.read_csv("four_test_data.csv")
valid_data = pd.read_csv("four_valid_data.csv")

In [3]:
#NA값 제거 하는중?
#토크나이징이랑 stopwords제거 안했는데 na값이 있네요?
train_data = train_data.dropna()
test_data = test_data.dropna()
valid_data = valid_data.dropna()

In [4]:
# split the dataset into training and validation datasets 
train_x, train_y = train_data['text'], train_data['label']
valid_x, valid_y = valid_data['text'], valid_data['label']
# 라벨을 숫자로 인코딩 하는 과정
# happy -> 0, sad -> 1 이런식으로 뭐가 뭘로 바뀔지는 모름
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [5]:
#count_vect.fit(train_data['text']) 에러 나는데 같이 풀어야 할 것 같네요....
#ValueError: np.nan is an invalid document, expected byte or unicode string.
#스택 오버 플로우에 하라는데로 했는데도 안됨....
#이거 되면 나이브 베이즈 바로 될 수도....
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(train_data['text'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [6]:
#이건 tf - idf 벡터라이저입니다.
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=15000)
tfidf_vect.fit(train_data['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

In [7]:
# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=15000)
tfidf_vect_ngram.fit(train_data['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=15000)
tfidf_vect_ngram_chars.fit(train_data['text'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

In [8]:
#Stopwords 만들게요....
stop_words_set = pd.read_csv('stopwords100.txt',header = 0, delimiter = '\t', quoting = 3)
stop_words= (list(stop_words_set['aa']))
stop_words2 = ['은', '는', '이', '가', '하', '아', '것', '들','의', '있', '되', '수', '보', '주', '등', '한']
stop_words.extend(stop_words2)

In [9]:
#토크나이저 만들게요.... 책 참고.
from soynlp.tokenizer import MaxScoreTokenizer
from soynlp.normalizer import *
import re
from konlpy.tag import Okt

def preprocessing(text, okt, remove_stopwords = False, stop_words = []):
    text = only_hangle(text)
    text = repeat_normalize(text, num_repeats = 2)
    
    text_token = okt.morphs(text, stem = True)
    
    if remove_stopwords:
        text_token = [token for token in text_token if not token in stop_words]
        
    return text_token

In [10]:
#Train data & Valid data를 전처리 하겠습니다
#책에서 나온거 그대로
okt = Okt()
clean_train_data = []
clean_valid_data = []
clean_test_data = []
for text in train_data['text']:
    if type(text) == str:
        clean_train_data.append(preprocessing(text, okt, remove_stopwords = True, stop_words = stop_words))
    else:
        clean_train_data.append([])
        
for text in valid_data['text']:
    if type(text) == str:
        clean_valid_data.append(preprocessing(text, okt, remove_stopwords = True, stop_words = stop_words))
    else:
        clean_valid_data.append([])
        
for text in test_data['text']:
    if type(text) == str:
        clean_test_data.append(preprocessing(text, okt, remove_stopwords = True, stop_words = stop_words))
    else:
        clean_test_data.append([])

In [11]:
#잘됐나 테스트 해볼게요
clean_train_data[:4]
#clean_valid_data[:4]
#clean_test_data[:4]

[['집안', '어른', '동성동본', '결혼', '하다', '족보', '를', '갈아엎다', '등본', '떼다', '전주최씨', '자다'],
 ['진짜', '많이', '빠지다', 'ㅠㅠ', '아이', '홀', '라인', '거의', '쌍꺼풀', '돼다'],
 ['알다'],
 ['음', '아마', '아침', '드라마', '에서', '보다', '배우', '장', '선영', '이다', '배우', '미소']]

In [12]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_train_data)
train_sequences = tokenizer.texts_to_sequences(clean_train_data)
valid_sequences = tokenizer.texts_to_sequences(clean_valid_data)
test_sequences = tokenizer.texts_to_sequences(clean_test_data)

word_index = tokenizer.word_index

MAX_SEQUENCE_LENGTH = 70

In [13]:
#워드 임베딩으로 조져 볼게요
#wiki.ko.vec 오픈할 때는 경로 설정해주셔야 해요.
embeddings_index = {}
for i, line in enumerate(open('wiki.ko.vec')):
    values = line.split()
    embeddings_index[values[0]] = numpy.asarray(values[1:], dtype='float32')
    
    
embedding_matrix = numpy.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector    

In [14]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)

    return metrics.accuracy_score(predictions, valid_y)

In [15]:
# Naive Bayes on Count Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
print("NB, Count Vectors: ", accuracy)

# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
print("NB, WordLevel TF-IDF: ", accuracy)

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("NB, N-Gram Vectors: ", accuracy)

# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("NB, CharLevel Vectors: ", accuracy)

NB, Count Vectors:  0.5066595631326585
NB, WordLevel TF-IDF:  0.5024862369028592
NB, N-Gram Vectors:  0.36219144024152017
NB, CharLevel Vectors:  0.5273486059314509


In [16]:
# Linear Classifier on Count Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_count, train_y, xvalid_count)
print("LR, Count Vectors: ", accuracy)

# Linear Classifier on Word Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print("LR, WordLevel TF-IDF: ", accuracy)

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("LR, N-Gram Vectors: ", accuracy)

# Linear Classifier on Character Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("LR, CharLevel Vectors: ", accuracy)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LR, Count Vectors:  0.5065707689575564
LR, WordLevel TF-IDF:  0.5026638252530634
LR, N-Gram Vectors:  0.3799502752619428
LR, CharLevel Vectors:  0.5555851536139229


In [17]:
# SVM on Ngram Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("SVM, N-Gram Vectors: ", accuracy)

/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVM, N-Gram Vectors:  0.275883502042266


In [18]:
# RF on Count Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xvalid_count)
print("RF, Count Vectors: ", accuracy)

# RF on Word Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf, train_y, xvalid_tfidf)
print( "RF, WordLevel TF-IDF: ", accuracy)

/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RF, Count Vectors:  0.43775528325341856


/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RF, WordLevel TF-IDF:  0.44024152015627777
